Unique overlap between sequence 0 and 1: [9]
Unique overlap between sequence 0 and 2: []
Unique overlap between sequence 1 and 2: [8]
Remaining elements of sequence 0: [1, 3, 4, 5, 7]
Remaining elements of sequence 1: [2, 5, 7]
Remaining elements of sequence 2: [5, 7]


In [1]:
def find_unique_overlaps(sequences):
    # Convert all index strings to sets to remove duplicates and for set operations
    sets = [set(seq) for seq in sequences]
    num_seqs = len(sets)
    unique_overlaps = []
    remaining_parts = []
    overlap_matrix_len = [[None for _ in range(num_seqs)] for _ in range(num_seqs)]
    overlap_matrix_index = [[None for _ in range(num_seqs)] for _ in range(num_seqs)]

    # Calculate overlaps for all possible pair combinations
    for i in range(len(sets)):
        remaining = sets[i].copy()  # Copy the current set to track remaining parts
        for j in range(i+1, len(sets)):
            # Find the intersection of two sets
            overlap = sets[i] & sets[j]
            
            # Eliminate elements that have already overlapped
            unique_overlap = set()
            for item in overlap:
                if all(item not in s for k, s in enumerate(sets) if k not in [i, j]):
                    unique_overlap.add(item)

            # Record the size of elements that only overlap once in an upper triangular matrix
            overlap_size = len(unique_overlap)
            overlap_matrix_len[i][j] = overlap_size 
            overlap_matrix_index[i][j] = unique_overlap

            # Add elements that only overlap once to the result list
            unique_overlaps.append(((i, j), unique_overlap))

            # Remove this overlap from both the current set and the corresponding set
            sets[i] -= unique_overlap
            sets[j] -= unique_overlap
            remaining -= unique_overlap

        # Add the remaining parts
        remaining_parts.append((i, remaining))
    
    # Set diagonal entries to the size of the remaining parts of each sequence
    for i in range(len(sets)):
        overlap_matrix_len[i][i] = len(remaining_parts[i][1])
        overlap_matrix_index[i][i] = remaining_parts[i][1]

    # Return unique overlaps, remaining parts of each sequence, and the upper triangular matrices
    return unique_overlaps, remaining_parts, overlap_matrix_len, overlap_matrix_index


# 示例使用
index_strs = [
    [1, 3, 4, 5, 7, 9],
    [2, 5, 7, 8, 9],
    [5, 7, 8],
    [1,0,3]
]
## assign a feature has 10 values torchsize 10,100
import torch
feature = torch.randn(10,2)
print(feature)

overlaps, remainings, overlap_matrix,overlap_matrix_index = find_unique_overlaps(index_strs)

for idx_pair, unique_overlap in overlaps:
    print(f"Unique overlap between sequence {idx_pair[0]} and {idx_pair[1]}: {sorted(list(unique_overlap))}")

for idx, remaining in remainings:
    print(f"Remaining elements of sequence {idx}: {sorted(list(remaining))}")

print("Overlap matrix (upper triangular):")
for row in overlap_matrix:
    print(row)
import numpy as np
import numpy as np

def compute_cumulative_upper_triangular(matrix):
    matrix_np = np.array(matrix, dtype=np.object)
    matrix_np[matrix_np == None] = 0
    matrix_np = matrix_np.astype(int)

    cumulative_matrix = np.zeros_like(matrix_np)
    for i in range(matrix_np.shape[0]):
        for j in range( matrix_np.shape[1]):
            cumulative_matrix[i][j] = matrix_np[i][j]
            if j > 0:
                cumulative_matrix[i][j] += cumulative_matrix[i][j-1]
            if i>0 and j== 0:
                cumulative_matrix[i][j] += cumulative_matrix[i-1][-1]
           

    return cumulative_matrix

cumulative_matrix = compute_cumulative_upper_triangular(overlap_matrix)-1
print("Cumulative matrix (upper triangular):")
for row in cumulative_matrix:
    print(list(row))
# Function to extract features based on the overlap index matrix
def extract_features(overlap_matrix_index, feature):
    # Create a list to hold the selected features
    selected_features = []

    # Convert the PyTorch tensor to a NumPy array
    feature_np = feature.numpy()

    # Traverse the index matrix of the upper triangular matrix
    for i in range(len(overlap_matrix_index)):
        for j in range(i, len(overlap_matrix_index[i])):
            if overlap_matrix_index[i][j] is not None:
                # Iterate through each index in the set
                for index in overlap_matrix_index[i][j]:
                    # Check if the index is within a valid range
                    if index < len(feature_np):
                        # Retrieve the feature
                        selected_features.append(feature_np[index])

    # Convert the list to a numpy array
    return np.vstack(selected_features)

# Example usage
extracted_features = extract_features(overlap_matrix_index, feature)
print(extracted_features)
print(extracted_features.shape)



tensor([[-0.7476,  1.3151],
        [-0.7508,  1.2394],
        [ 2.2277,  0.8517],
        [-0.3507,  0.5771],
        [ 0.0650, -0.2427],
        [-1.2542, -0.7449],
        [-0.6801, -0.5727],
        [-0.7353, -0.5938],
        [-0.2132,  1.2257],
        [-0.4303, -0.8708]])
Unique overlap between sequence 0 and 1: [9]
Unique overlap between sequence 0 and 2: []
Unique overlap between sequence 0 and 3: [1, 3]
Unique overlap between sequence 1 and 2: [8]
Unique overlap between sequence 1 and 3: []
Unique overlap between sequence 2 and 3: []
Remaining elements of sequence 0: [4, 5, 7]
Remaining elements of sequence 1: [2, 5, 7]
Remaining elements of sequence 2: [5, 7]
Remaining elements of sequence 3: [0]
Overlap matrix (upper triangular):
[3, 1, 0, 2]
[None, 3, 1, 0]
[None, None, 2, 0]
[None, None, None, 1]
Cumulative matrix (upper triangular):
[2, 3, 3, 5]
[5, 8, 9, 9]
[9, 9, 11, 11]
[11, 11, 11, 12]
[[ 0.06504019 -0.24265988]
 [-1.2542088  -0.7448714 ]
 [-0.7353464  -0.5937597 ]


/tmp/ipykernel_173519/4063231807.py:75: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  matrix_np = np.array(matrix, dtype=np.object)


In [93]:
print("Overlap matrix (upper triangular):")
for row in overlap_matrix_index:
    print(row)

# Fill out the symmetric matrix
for i in range(len(overlap_matrix_index)):
    for j in range(len(overlap_matrix_index[i])):
        # Copy the elements from the upper triangular part to the symmetric position
        if overlap_matrix_index[i][j] is  None:
            overlap_matrix_index[i][j] = overlap_matrix_index[j][i]
## print the overlap_matrix_index
print("Overlap matrix (symmetric):")
for row in overlap_matrix_index:
    print(row)
# Initialize the local_index_matrix as a square matrix filled with None
local_index_matrix = [[None for _ in range(len(overlap_matrix_index))] for _ in range(len(overlap_matrix_index))]

for i in range(len(overlap_matrix_index)):
    for j in range( len(overlap_matrix_index[i])):
        if overlap_matrix_index[i][j] is not None:
            # Find the local indices of the overlap_matrix_index[i][j] in index_strs
            local_indices = [index_strs[i].index(x) for x in overlap_matrix_index[i][j]]
            local_index_matrix[i][j] = local_indices
            # Since the matrix is symmetric, set the corresponding lower triangular element
           

print("local_index_matrix:") 
for row in local_index_matrix:
    print(row)


Overlap matrix (upper triangular):
[{4, 5, 7}, {9}, set(), {1, 3}]
[None, {2, 5, 7}, {8}, set()]
[None, None, {5, 7}, set()]
[None, None, None, {0}]
Overlap matrix (symmetric):
[{4, 5, 7}, {9}, set(), {1, 3}]
[{9}, {2, 5, 7}, {8}, set()]
[set(), {8}, {5, 7}, set()]
[{1, 3}, set(), set(), {0}]
local_index_matrix:
[[2, 3, 4], [5], [], [0, 1]]
[[4], [0, 1, 2], [3], []]
[[], [2], [0, 1], []]
[[0, 2], [], [], [1]]


In [94]:
def query_features_by_sequence(cumulative_matrix, extracted_features):
    num_sequences = len(cumulative_matrix)
    feature_ranges_by_sequence = []
    all_features = []
    for i in range(num_sequences):
        feature_ranges = []
        feature_i=torch.zeros(len(index_strs[i]),2)
        for j in range(num_sequences):
            print(i,j)
            if j >= i:
                start_idx = cumulative_matrix[i][j-1]+1 if j != 0 else 0
                end_idx = cumulative_matrix[i][j] +1
            else:
                start_idx = cumulative_matrix[j][i-1]+1 
                end_idx = cumulative_matrix[j][i] +1
            # Assuming extracted_features is a NumPy array and local_index_matrix is properly defined
            feature_range = extracted_features[start_idx:end_idx]
            if start_idx == end_idx:
                feature_range = np.array([])  # Ensure feature_range is a NumPy array even when empty
            if len(feature_range) > 0 and feature_range.size > 0:
                # Convert feature_range to a PyTorch tensor and assign it to the correct index in feature_i
                feature_i[local_index_matrix[i][j]] = torch.from_numpy(feature_range)
        all_features.append(feature_i)
    return all_features

sequence_feature = query_features_by_sequence(cumulative_matrix, extracted_features)

for i, features in enumerate(sequence_feature):
    print(f"Features for sequence {i}:")
    print(features)
print(feature)

0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3
Features for sequence 0:
tensor([[ 0.9389,  0.9274],
        [-0.5641, -1.8868],
        [ 1.7817, -0.8116],
        [ 0.2932, -0.5811],
        [-0.6809, -0.8817],
        [-1.3227,  2.7849]])
Features for sequence 1:
tensor([[ 1.2467, -0.7938],
        [ 0.2932, -0.5811],
        [-0.6809, -0.8817],
        [ 0.7802,  1.2738],
        [-1.3227,  2.7849]])
Features for sequence 2:
tensor([[ 0.2932, -0.5811],
        [-0.6809, -0.8817],
        [ 0.7802,  1.2738]])
Features for sequence 3:
tensor([[ 0.9389,  0.9274],
        [ 0.2601, -0.5790],
        [-0.5641, -1.8868]])
tensor([[ 0.2601, -0.5790],
        [ 0.9389,  0.9274],
        [ 1.2467, -0.7938],
        [-0.5641, -1.8868],
        [ 1.7817, -0.8116],
        [ 0.2932, -0.5811],
        [ 1.2158,  0.0988],
        [-0.6809, -0.8817],
        [ 0.7802,  1.2738],
        [-1.3227,  2.7849]])


In [1]:
import matplotlib.pyplot as plt
import networkx as nx

# Define a tree structure using nested tuples
tree_structure = (
    ('G1', ('G1',), ('G2',), ('G3',), ('G4',)),
    ('G2', ('G3',), ('G4',)),
    ('G3', ('G4',)),
    ('G4',),
)

# Create a directed graph object
G = nx.DiGraph()

# Function to add nodes and edges to the graph
def add_edges(tree, parent=None):
    # base case: if the tree is a string, we've hit a leaf node, return
    if isinstance(tree, str):
        return
    # recursive case: otherwise, the tree is a node with children
    node, children = tree[0], tree[1:]
    G.add_node(node)  # add the node
    if parent:
        G.add_edge(parent, node)  # add an edge to the parent
    for child in children:
        add_edges(child, node)  # add edges to children recursively

# Add nodes and edges for each nested tuple
for branch in tree_structure:
    add_edges(branch)

# Draw the graph
pos = nx.drawing.nx_agraph.graphviz_layout(G, prog='dot')
nx.draw(G, pos, with_labels=True, arrows=True)

# Save the figure
plt.savefig('tree_structure.png')
plt.show()


ImportError: requires pygraphviz http://pygraphviz.github.io/